<a href="https://www.kaggle.com/code/dibkb9/efficientnetb0-with-fine-tuning-accuracy-95?scriptVersionId=155978763" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_path = '/kaggle/input/smoking/Training/Training'
test_path = '/kaggle/input/smoking/Testing/Testing'
valid_path = '/kaggle/input/smoking/Validation/Validation'

In [3]:
def extract_from_path(path)-> []:
    full_path = []
    for i in sorted(os.listdir(path)):
        full_path.append(os.path.join(path, i))
    return full_path

In [4]:
train_images = extract_from_path(train_path)
test_images = extract_from_path(test_path)
valid_images = extract_from_path(valid_path)

In [5]:
def createDataFrame(image_paths):
    labels = [path.split('/')[-1].split('_')[0] for path in image_paths]
    df = pd.DataFrame({'path': image_paths, 'label': labels})
    df['id'] = df['label'].apply(lambda x: 1 if x == 'smoking' else 0)
    return df

In [6]:
train_images_df = createDataFrame(train_images)
test_images_df = createDataFrame(test_images)
valid_images_df = createDataFrame(valid_images)

In [7]:
import tensorflow as tf

IMAGE_SIZE = (224,224)
BATCH_SIZE = 32
SEED = 42

def imgPreProcessing(image, label):
    img = tf.io.read_file(image)
    img = tf.io.decode_jpeg(img, channels = 3)
    img = tf.image.resize(img, size = (IMAGE_SIZE))
    return img, label
def loadDataset(df:pd.DataFrame):
    dataset = tf.data.Dataset.from_tensor_slices((df['path'], df['id']))
    return (dataset
                 .map(imgPreProcessing)
                .shuffle(BATCH_SIZE * 20)
                 .batch(BATCH_SIZE))

In [8]:
train_data = loadDataset(train_images_df)
test_data = loadDataset(test_images_df)
valid_data = loadDataset(valid_images_df)

##### Fitting a baseline model on the dataset (EfficientNetB0)

In [9]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import efficientnet_v2
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

# Define image size
IMAGE_SIZE = (224, 224)

# Create the EfficientNetV2B0 base model
base_model = efficientnet_v2.EfficientNetV2B0(include_top=False)
base_model.trainable = False

# Build the Keras Sequential model
model = Sequential([
    Input(shape=IMAGE_SIZE + (3,)),
    base_model,
    GlobalAveragePooling2D(),
    Dense(2, activation="sigmoid")
])

# Compile the model
model.compile(
    loss=SparseCategoricalCrossentropy(),
    optimizer=Adam(),
    metrics=["accuracy"]
)

# Display model summary
model.summary()

24274472/24274472 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-b0 (Functio  (None, None, None, 1280   5919312   
 nal)                        )                                   
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 5921874 (22.59 MB)
Trainable params: 2562 (10.01 KB)
Non-trainable params: 5919312 (22.58 MB)
_________________________________________________________________


In [10]:
initial_epochs = 8
model.fit(
                                                    train_data,epochs = initial_epochs, 
                                                    steps_per_epoch = len(train_data),
                                                    validation_data = valid_data, 
                                                    validation_steps = len(valid_data),
                                                  )

Epoch 1/8


2023-12-21 18:12:19.370859: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/efficientnetv2-b0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


23/23 [==============================] - 25s 237ms/step - loss: 0.5891 - accuracy: 0.6830 - val_loss: 0.3426 - val_accuracy: 0.8833
Epoch 2/8
23/23 [==============================] - 2s 69ms/step - loss: 0.3977 - accuracy: 0.8352 - val_loss: 0.2436 - val_accuracy: 0.9111
Epoch 3/8
23/23 [==============================] - 2s 71ms/step - loss: 0.3412 - accuracy: 0.8575 - val_loss: 0.2080 - val_accuracy: 0.9278
Epoch 4/8
23/23 [==============================] - 2s 69ms/step - loss: 0.2991 - accuracy: 0.8771 - val_loss: 0.2157 - val_accuracy: 0.9167
Epoch 5/8
23/23 [==============================] - 2s 68ms/step - loss: 0.2814 - accuracy: 0.8994 - val_loss: 0.1970 - val_accuracy: 0.9222
Epoch 6/8
23/23 [==============================] - 2s 69ms/step - loss: 0.2602 - accuracy: 0.9022 - val_loss: 0.1894 - val_accuracy: 0.9167
Epoch 7/8
23/23 [==============================] - 2s 69ms/step - loss: 0.2484 - accuracy: 0.9134 - val_loss: 0.1931 - val_accuracy: 0.9278
Epoch 8/8
23/23 [===========

In [11]:
import os
import glob
files = glob.glob('tfjs_dir/*')
for f in files:
    os.remove(f)

In [12]:
# os.rmdir("tfjs_dir")

In [13]:
os.mkdir("tfjs_dir")
# convert to Tensorflow.js
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, 'tfjs_dir')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:
os.listdir('tfjs_dir')

['group1-shard6of6.bin',
 'group1-shard3of6.bin',
 'group1-shard1of6.bin',
 'model.json',
 'group1-shard4of6.bin',
 'group1-shard2of6.bin',
 'group1-shard5of6.bin']

#### ⚡️ Data Augmentaion

In [15]:
# view the files that make up the tensorflow.js model
